In [ ]:
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt
import sys
import os
import random
from importlib import reload
from tqdm import tqdm
import copy

# Generate Data

In [ ]:
# make data - using uniform dist and ranges
def generate_data(data_ranges, length, epsilon=0.0):
    # get max dims
    dims_x = [np.inf, -np.inf]
    dims_y = [np.inf, -np.inf]
    for clus in data_ranges:
        if(clus[0][0] < dims_x[0]):
            dims_x[0] = clus[0][0]
        if(clus[0][1] > dims_x[1]):
            dims_x[1] = clus[0][1]
        if(clus[1][0] < dims_y[0]):
            dims_y[0] = clus[1][0]
        if(clus[1][1] > dims_y[1]):
            dims_y[1] = clus[1][1]

    # generate
    data = np.zeros((1,2))
    labels = []
    for i,clus in enumerate(data_ranges):
        for j in range(length):
            if(np.random.rand() >= epsilon):
                data = np.append(data, [[
                    np.random.uniform(clus[0][0], clus[0][1]),
                    np.random.uniform(clus[1][0], clus[1][1]),
                ]], axis=0)
                labels.append(i)
            else:
                data = np.append(data, [[
                    np.random.uniform(dims_x[0], dims_x[1]),
                    np.random.uniform(dims_y[0], dims_y[1])
                ]], axis=0)
                labels.append(9)

    data = data[1:]

    return data, labels

In [ ]:
# Split data for display - plt
def split_data(data, labels):
    data_sep = []
    uniq_labels = list(set(sorted(labels)))
    for cl in uniq_labels:
        data_class = np.zeros((1,2))
        for i,lab in enumerate(labels):
            if(cl == lab):
                data_class = np.append(data_class, [data[i]], axis=0)
        data_class = data_class[1:]
        data_sep.append(data_class)
    return data_sep, uniq_labels

In [ ]:
# graph separated data w/ plt
def plt_graph_sep_data(data):
    fig, ax = plt.subplots()
    for cl in data:
        ax.scatter(cl[:,0], cl[:,1])
    plt.show()

In [ ]:
def expand_labels(labels):
    uniqs = list(set(sorted(labels)))
    expanded = []
    for lab in labels:
        new_exp = [0] * len(uniqs)
        new_exp[uniqs.index(lab)] = 1
        expanded.append(new_exp)
    return expanded

In [ ]:
data_ranges = [
    [ [-14.0, -2.0], [6.0, 16.0] ],
    [ [2.0, 15.0],   [-8.0, 6.0] ],
    [ [-12.0, -4.0], [-14.0, -6.0] ]
]
data, labels = generate_data(data_ranges, length=100)

# Train RBN

In [ ]:
labels_expand = expand_labels(labels)

In [ ]:
import rbn
reload(rbn)

rbnetwork = rbn.RBN(k=50, outputs=1)
error, centers = rbnetwork.train(data, labels, alpha=50e-1, epochs=600, batch_size=100, error_target=.00005)

# rbnetwork = rbn.RBN(k=50, outputs=3)
# error = rbnetwork.train(data, labels_expand, alpha=1e-1, epochs=500, batch_size=100, error_target=1e-2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(error)

# Test RBN (resub)

In [ ]:
# new_data = np.array([*data[0:2], *data[100:102], *data[200:202]])
# print(new_data.shape)
truepos = 0
preds = []
for d, l in zip(data, labels):
    pred = rbnetwork.forward(d)
    pred = round(pred[0][0])
    preds.append(pred)
    if(pred == l):
        truepos += 1

accuracy = truepos / len(data)
print('Accuracy:', round(accuracy,2))

In [ ]:
data_sep, uniq_labels = split_data(data, preds)
data_sep.append(centers)
plt_graph_sep_data(data_sep)

# Test kmeans

In [ ]:
import kmeans
reload(kmeans)

km = kmeans.KMeans(k=100)
centers = km(data, eta=.1, error_target=.001)

fig, ax = plt.subplots()
ax.scatter(data[:,0], data[:,1])
for cen in centers:
    ax.scatter(*cen)